# **MADRID - MetAbolic Drug Repurposing and IDentification** 

## **Who should use this pipeline?**

This Jupyterlab pipeline has everything necessary to build a context-specific constraint-based metabolic model (CBMM) from any single source or combination of sources from the following '-omics' data.
- Bulk RNA-seq
- Single-cell RNA-seq
- Proteomics (specific name???)
- Microarray

It also serves as a platform to use these models to identify drug targets and potentially repurposable drugs for metabolism-impacting diseases.

MADRID does not require any amount of programming experience to create models. However every step of the pipeline is packaged in its own .py file to promote accessible modification, addition, or replacement of analysis steps. The Jupyterlab container comes pre-loaded with the most popular R and Python libraries, however, if you would like to use a library and cannot install it for any reason, please request it on our Github page!<br>
https://github.com/HelikarLab/MADRID

In this pipeline, **the term "context" refers to a specific state that can be subset from a genome-scale metabolic model using experimental data.** This context can be a specific type of cell, or tissue in a specific experimental state such as a control or a disease. For drug perturbation scoring of a specific cell-type or tissue, **it is only necessary to build a CBMM (Steps 1-5) for the healthy control state.** Differential gene expression (Step 6) will be used to for disease analysis so that multiple diseaes can be analyzed using the same CBMM.

***Warning! If you terminate your session after running the Docker, any changes you make WILL NOT BE SAVED! Please mount a local directory to the Docker image as instructed on the Github and Dockerhub README's to prevent data loss.**


## **Before You Start**

### **Before running, you must provide the proper files depending on the types of data you will use for model creation.**
- For RNA-seq: Either a correctly formatted folder named "MADRID_input" in the the data directory. Proper inputs can be generated using our Snakemake pipeline specifically designed for MADRID: https://github.com/HelikarLab/FastqToGeneCounts.  RNA-seq data can either be single-cell or bulk, but our Snakemake pipeline provided is for bulk only at the time. If processing RNA-seq data with an alternative procedure, or importing a pre-made gene count matrix, follow the instructions in Step 1.

- For proteomics. A matrix of measurement ****NEED TO DISCUSS WITH BHANWAR WHAT THESE VAULES MEAN***** where rows are proteins in Entrez format and columns are arbitrary sample names.

- For microarray, results must be uploaded to Gene Expression Omnibus, the only thing to provide in MADRID is a configuration file with GSE, GSM, and GPL codes. microarray_data_inputs.xlsx shows a template to use. Note that microarray has become obsolete and it is recomended to use RNA-seq if possible.

### **Six steps to identifying drug targets, stop after step 3 if building a context-specific model for other purposes**
        
1. Preprocess Bulk RNAseq data by converting STAR outputed gene counts files into a unified matrix, fetch necessary info about each gene required for normalization, and generate a configuration sheet.

2. Analyze any combination of microarray, RNAseq (total, polyA, single-cell), and proteomics data, output a list of active genes for each strategy.

3. Check for consensus amongst strategies according to desired rigor and merge into a singular set of active genes

4. Create tissue specific models based on the list of active genes. If required the user can manually refine these models and supply them in Step 4. 

5. Identify differential gene expressions from disease datasets using either microarray or bulk RNAseq transcriptomics information.

6. Identify drug targets and repruposable drugs. This step consists of four substeps. 
    - mapping drugs on automatically created or user-supplied models
    - knock-out simulation
    - compare simulation results of perturbed and unperturbed models
    - integrate with disease genes and score drug targets.

### **Configuration sheet information**
The user should upload config .xlsx files to `/work/data/config_sheets`. The sheet names in these config files should correspond to the context (tissue name, cell name, control, etc.) where each sheet contains the sample names to include in that context-specifc model. These sample names should correspond to the sample (column) names in the source data matrix which should be uploaded (or outputed) in `/work/data/data_matrices/<model name>/`
    
In the original Docker image, some exemplary input files are included to build metabolic models of naive, Th1, Th2, and Th17 subtypes and identify drug targets for rheumatoid arthritis. User can follow the documentation and the format of the exemplary input files and and use the provided template files to create your own input files.

## **Step 1: Initialize and Preprocess RNA-seq data**
**(Skip if not using RNA-seq)**

RNA-seq data is read by MADRID as count matrix where each column is a different sample/replicate named 'exampleTissueName_SXRYrZ' where:
- X is the study (or batch) number
- Y is the replicate number
- Z is the run number. If the replicate does not contain multiple runs for a single replicate, then "rZ" should be neglected.
- exampleTissueName is the name of the model that will be built from this data. It should be consistant with other data sources if they are to be integrated. **Note that this identifier should not have any special characters including "\_" since it may interfere with parsing.**
<br>

Replicates should come from the same study/batch group and different study/batch numbers can come from different published studies (or batches) as long as the tissue/cell was under similar enough conditions for your modeling purpose. Run numbers in the same replicate will be summed together.
<br>
<br>
#### **Example:**
Say S1 represents a study or batch, and S2 represents a different study or batch of RNA-seq data from m0 macrophages whose model we will name m0Macro. The studies were conducted in a different lab, by different researchers at different times using a different library preparation kit. In each study there are two replicates (R1, and R2) obtained for each one. m0Macro_S1R1 and m0Macro_S1R2 will be checked for consensus to generate a list of active genes in both replicates. These active genes will then be checked for consensus with the consensus of m0Macro_S2R1 and m0Macro_S2R2 to output a list of active genes in both studies.
<br><br>
The reason this system is used is not only to help keep you and MADRID organized. Most types of normalized gene counts are not good for direct comparisons across replicates, and are especially not suitable for comparisons across different experiments. Therefore, MADRID will convert normalized gene counts into a boolean list of active genes. These lists will be compared at the the level of replicates in a batch (or study) and then again at the level of all provided batches (or study). Finally, the active genes will be merged with the outputs of proteomics, microarray and differnt RNA-seq stategies if provided. The rigor used at each level can be easily modified by the user.
<br>

#### **Two ways to initialize RNA-seq data**

1. Import a properly formatted `MADRID_inputs` folder in the `data` directory. 
<br>

> **It is recommended that you use our Snakemake pipeline to align and create a properly formated MADRID_inputs folder.** The pipeline also runs a series of important quality control methods to help determine if any of the provided samples are not suitable for model creation. This pipeline can be found at https://github.com/HelikarLab/FastqToGeneCounts.

> Or, **if using your own alignment protocol**, follow this guide to create a `MADRID_inputs` folder.

> - The top level of the directory has seperate tissues/cells to create seperate models from. The next level must have a folder called `geneCounts` and optionally a `strandedness` folder. If using zFPKM normalization, there should also be two more folders called `layouts` and `fragmentSizes`. Inside each of these folders should be folders named SX (wherer X is an arbitrary user-defined number that replicates are associated with. 
<br>    

> - Inside these study number folders of `geneCounts` should be outputs of STAR aligner with using --quantMode GeneCounts. To help MADRID (and you!) stay organized, these outputs should be renamed `exampleTissueName_SXRYrZ.tab` where X is the study (or batch) number, Y is the replicate number, and Z is the run number. If the replicate does not contain multiple runs the rZ should be neglected. Replicates should come from the same study/sample group and different samples can come from different published studies (or batches) as long as the tissue/cell was under similar enough conditions for your modeling purpose.
<br>

> - Inside the study number folders of `strandedness` should be files named `exampleTissue_SXRYrZ_strandedness.txt`. These files must tell the strandedness of the the RNA-seq method used, and must contain one of the following text (and nothing else):
>> * NONE
>> * FIRST_READ_TRANSCRIPTION_STRAND
>> * SECOND_READ_TRANSCRIPTION_STRAND
<br>

> - Inside the study number folders of `layouts` should be files named `exampleTissue_SXRYrZ_layout.txt` where each file tells the layout of the library used, and must contain one of the following text (and nothing else):
>> * paired-end
>> * single-end
<br>

> - Inside the study number folders of `fragmentSizes` should be files named `exampleTissue_SXRYrZ_fragment_sizes.txt` and contain the output of RSeQC's RNA_fragment_size.py function.

> - Inside the study number folders of `prepMethods` should be files named `exampleTissue_SXRYrZ_prep_method.txt` where each file tells the library preparation strategy, which must be one of the following.
>> * total
>> * mRNA
<br><br>
where 'total' refers to Total RNA and mRNA refers to polyA enriched RNA. Note that these strategies serve only to differentiate the methods in the event that both are used to build a model. If a different library strategy is desired, you can use either one as a placeholder, or will very little Python knowledge, it is easy add a new strategy to `merge_xomics.py`.  

2. Import a properly formatted counts matrix in `/work/data/data_matrices/exampleTissue/gene_counts_matrix_exampleTissue.csv` where the rows are name exampleTissue_SXRY (note the lack of run number since runs should be summed into a single set of counts). **If providing the count matrix this way, instead of generating one using method 1, you will also have to create a configuration file** that has each sample's name, study/batch number, and if using zFPKM, layout and mean fragment length. **Use the provided template** to help. Once provided, run rnaseq_preprocess.py with the '--provide-matrix' argument. <br><br> ***This method is best if you are downloading a premade count matrix, or using single-cell data that has already been batch corrected, clustered, and sorted into only the cell type of interest!**

Currently, MADRID can filter raw RNA-seq counts using three normalization techniques.

> - TPM Quantile, where each replicate is normalized with Transcripts-per-million and an upper quantile is taken to create a boolean list of active genes for the replicate. Replicates are compared for consensus within the study/batch number according to user-defined ratios and then study/batch numbers are checked for consensus according to different user defined ratios. **Recomended if user wants more control over the size of the model, like a smaller model that allows for only the most expressed reactions, or a larger more encompassing one that contains less essential reactions.

> - zFPKM method outlined in: https://pubmed.ncbi.nlm.nih.gov/24215113/ can be used. Counts will be normalized using zFPKM and genes > -3 will be considered expressed per thier recommendation. Expressed genes will be checked for consensus at the replicate and study/batch levels the same as TPM Quantile. **Recommended if user wants to give least input over gene essentially determination and use the most standardized method of active gene determination. 

> - flat cutoff of CPM (counts per million) normalized values, check for consensus the same as other methods. **Not recommended**

Regardless of normalization technique used, or provided files used for RNA-seq, preprocessing is required to fetch relevent gene information needed for harmonization and normalization such as Entrez ID, and the start and end postions.


In [ ]:
# import necessary python packages
import sys
import os
import pandas as pd
import numpy as np
import json
import re
from subprocess import call
from project import configs
import bioservices
import pprint
import troppo

In [ ]:
# Step 1: Preprocess RNAseq data by generating a counts matrix, config sheet, and gene file from MADRID_inputs.

# tissue name or list of tissue names within a string
context_names = "naiveB smB"
create_counts_matrix = True  # set to false if using a pregenerated matrix file
gene_format = "Ensembl"      # accepts 'Entrez', 'Ensembl', and 'Symbol'
taxon_id = "human"           # accepts integer (bioDBnet taxon id) or 'human' or 'mouse'
preprocess_mode = "create-matrix" # "create-matrix" or "provide-matrix"

cmd = ' '.join(
    [
        'python3', 'rnaseq_preprocess.py',
        '--context-names', f'"{context_names}"',
        '--gene-format', f'"{gene_format}"',
        '--taxon-id', f'"{taxon_id}"',
        f'--{preprocess_mode}'
    ]
)

!{cmd}

## Step 2: Identifying Gene Activity in Transcriptomics and Proteomics Datasets

Identify gene activity in the following data sources 
 - RNA-seq (bulk or single-cell)
 - Proteomics
 - Microarray

 Only one sources is required for model generation, multiple can be helpful for additional validation if of high-quality. 

### Microarrays

From wikipedia: A microarray is a multiplex lab-on-a-chip. Its purpose is to simultaneously detect the expression of thousands of genes from a sample (e.g. from a tissue). It is a two-dimensional array on a solid substrate—usually a glass slide or silicon thin-film cell—that assays (tests) large amounts of biological material using high-throughput screening miniaturized, multiplexed and parallel processing and detection methods.

MADRID can directly download and analyze microarray data from GEO for Agilent and Affymetrix platoforms. Follow the template and example in `/work/data/config_sheets/` to use microarray data in your analysis.

Microarray technology is becoming increasingly obsolete, if possible it is recommended that you use RNA-seq instead. Although different strategies exist for microarrays, MADRID does not distinguish between them nor are  there any plans to in the future due to it's obsoelecense.

### **RNA-seq Analysis**

RNA-seq analysis has two primary types, bulk tissue, and single-cell. Bulk RNA-seq also has multiple strategies of library preparation. If using public data, the user may run into a situation where they wish to use a combination of bulk RNA-seq data produced using two very different library preparation strategies. 
  

#### **Bulk RNA-seq**

MADRID currently supports the two most common strategies, mRNA (polyA) enriched RNA-seq, and total RNA-seq. 

Because of expected differences in distribution of transcripts, MADRID is written to handle each strategy seperately before the integration step. The recommended Snakemake alignment pipeline is designed to work with MADRID's preprocessing step (Step 1) to split RNA-seq data from GEO into seperate input matrices and config sheets.

**To create a gene expression file for total RNA-seq data, use the "total" for the '--library-prep' argument**

**To create a gene expression file for mRNA enriched / polA RNA-seq data, use the "mRNA" for the '--library-prep' argument.**

The analysis of each strategy is identical so specifying the type only serves to ensure MADRID analyzes them seperately.

#### **Single-cell RNA-seq**

While the Snakemake pipeline does not yet support single-cell alignment and MADRID does not yet support automated configuration file and counts-matrix file creation for single-cell alignment output from STAR, it is possible to use single-cell data to create a model with MADRID. 

Since normalization strategies can be applied to single-cell the same way it is applied to bulk, rnaseq_gen.py can be used with a provided counts matrix and config sheet (see Step 1 to help create it). 

Just like 'total' and 'mRNA', rnaseq_gen.py can be run with "SC" as the '--library-prep' argument to help MADRID differentiate it from the bulk RNA-seq data if using multiple strategies.   

In [ ]:
# step 2.2 RNA-seq Analysis for Total RNA-seq library preparation

# config for total rna-seq
trnaseq_config_file = 'trnaseq_data_inputs_auto.xlsx'

rep_ratio = 0.75         # proportion of replicates for a gene to be active in a sample
group_ratio = 0.75       # proportion of samples with expression required for gene
rep_ratio_h = 1.0        # proportion of replicates with expression required for high-confidence
group_ratio_h = 1.0      # proportion of replicates with expression required for high-confidence
technique = "zFPKM"      # quantile, cpm, or zfpkm
quantile = 50            # cutoff TPM quantile for quantile filtering
min_zfpkm = -3           # cutoff for CPM filtering
prep_method = "total"    # library prepartion method ('total', mRNA', or 'SC')

cmd = ' '.join(
    [
        'python3', 'rnaseq_gen.py',
        '--config-file', f'"{trnaseq_config_file}"',
        '--replicate-ratio', f'"{rep_ratio}"',
        '--batch-ratio', f'"{group_ratio}"',
        '--high-replicate-ratio', f'"{rep_ratio_h}"',
        '--high-batch-ratio', f'"{group_ratio_h}"',
        '--filt-technique', f'"{technique}"',
        '--min-zfpkm', f'"{min_zfpkm}"',
        '--library-prep', f'"{prep_method}"'
    ]
)

!{cmd}

In [ ]:
 # step 2.3 mRNA capture (polyA) RNA-seq Analysis

# config for mRNA (polyA) enriched RNA-seq
mrnaseq_config_file = 'mrnaseq_data_inputs_auto.xlsx'

rep_ratio = 0.75         # proportion of replicates for a gene to be active in a sample
group_ratio = 0.75       # proportion of samples with expression required for gene  
rep_ratio_h = 1.0        # proportion of replicates with expression required for high-confidence
group_ratio_h = 1.0      # proportion of replicates with expression required for high-confidence
technique = "zfpkm"      # quantile-tpm, cpm, or zfpkm
min_zfpkm = -3           # cutoff for CPM for filtering
prep_method = "mrna"     # library preparation method

cmd = ' '.join(
    [
        'python3', 'rnaseq_gen.py',
        '--config-file', f'"{mrnaseq_config_file}"',
        '--replicate-ratio', f'"{rep_ratio}"',
        '--batch-ratio', f'"{group_ratio}"',
        '--high-replicate-ratio', f'"{rep_ratio_h}"',
        '--high-batch-ratio', f'"{group_ratio_h}"',
        '--filt-technique', f'"{technique}"',
        '--min-zfpkm', f'"{min_zfpkm}"',
        '--quantile', f'"{quantile}"',
        '--library-prep', f'"{prep_method}"'
    ]
)
                
!{cmd}

In [ ]:
# Step 2.5 Proteomics Analysis

# config file for proteomics
proteomics_config_file = 'proteomics_data_inputs_paper.xlsx'

# ratio of replicates required for a gene to be considered active in that sampler
rep_ratio = 0.75
batch_ratio = 0.75

# Genes can be considered high confidence if they are expressed in a high proportion of samples.
# High confidence genes will be considered expressed regardless of agreement with other data sources
high_rep_ratio = 1.0
high_batch_ratio = 1.0
quantile = 25

cmd = ' '.join(
    [
        'python3', 'proteomics_gen.py',
        '--config-file', f'"{proteomics_config_file}"',
        '--replicate-ratio', f'"{rep_ratio}"',
        '--high-replicate-ratio', f'"{high_rep_ratio}"',
        '--batch-ratio', f'"{batch_ratio}"',
        '--high-batch-ratio', f'"{high_batch_ratio}"',
        '--quantile', f'"{quantile}"'
    ]
)

!{cmd}

## Step 3: Merge Expression from Different Data Sources

So far, active genes have been determined for at least one data source. If using multiple sources of any combination of microarray, bulk RNA-seq of either total RNA or mRNA capture (polyA), proteomics, or single-cell RNA-seq. Now we can merge the active genes from each data source to make a list of active genes that is more comprehenisve or more strict than any individual list. 

`merge_xomics.py` takes each data source discussed so far as an argument, and it is easy to add new ones to the script if desired. The other arguments to consider are:
- **--expression-requirement** which is the number of data souces with expression required for a gene to be considered active if not a high-confidence gene for any source (defaults to the total number of input data sources arguments provided)

- **--requirement-adjust** is the method to adjust this requirement in the event that tissues have different numbers of provided data sources. (*Note that this does nothing if there is only one tissue type in the config files). 

> - "progressive" - expression requirement applies to tissue(s) with lowest number of data source types. Tissues with more will require 1 more source to have an active gene per additional source provided for the gene to be active in the model.
        
> - "regressive" - expression requirement applies to the tissue(s) with largest number of data source types. Tissues with less will require 1 less source to have an active gene per missing source for the gene to be active in the model
                    
> - "flat" - (default) expression requirement is used regardless of differences in number of data sources provided for different tissues

> - "custom" - (Not yet implemented!) an .xlsx file where column one is the tissue type, and column two is the expression requirement. Requires additional argument '--requirements-file' whose value is the filename of this .xlsx file in `/work/data/`
                          
- **--no-hc** use this flag to prevent high-confidence gene from overiding expression_requirement

- **--no-na-adjustment** use this flag to prevent genes that are not a present in one data source, but are present in others from subtracting one from the expression requirement.


If the '--no-hc' flag is not used, any gene that was determined to be high-confidence in any input data source will cause the gene to be active in the final model, regardless of agreement with other sources.**
<br />

If the '--no-na-adjustment' flag is not used, any time a gene is NA in a data source, meaning it was not tested for in the library of that data source, but was tested in the library of at least one other, it will subtract one from the expression requirement.

***Adjusted expression requirement will never resolve to be < 1 or > the number of data sources that tissue is given**

In [ ]:
# Step 2.4 Merge the gene lists of data sources, create a list of active gene IDs
trnaseq_config_file = "trnaseq_data_inputs_auto.xlsx"
mrnaseq_config_file = "mrnaseq_data_inputs_auto.xlsx"
proteomics_config_file = "proteomics_data_inputs_paper.xlsx"

expression_requirement = 3
        
requirement_adjust = "regressive" 
tweight = 6
mweight = 6
scweight = 6
pweight = 10


cmd = ' '.join(
    [
        'python3', 'merge_xomics.py',
        '--merge-distribution',
        #'--microarray-config-file', f'"{microarray_config_file}"',  # If using micro-array, uncomment the start of this line
        '--total-rnaseq-config-file', f'"{trnaseq_config_file}"',
        '--mrnaseq-config-file', f'"{mrnaseq_config_file}"',
        #'--scrnaseq-config-file', f'"{scrnaseq_config_file}"',      # If using single-cell data, uncomment the start of this line
        '--proteomics-config-file', f'"{proteomics_config_file}"',
        '--expression-requirement', f'"{expression_requirement}"',
        '--requirement-adjust', f'"{requirement_adjust}"',
        '--total-rnaseq-weight', f'"{tweight}"',
        '--mrnaseq-weight', f'"{mweight}"',
        #'--single-cell-rnaseq-weight', f'"{scweight}"',             # If using single-cell data, uncomment the start of this line
        '--protein-weight', f'"{pweight}"',
        '--no-hc'
    ]
)

!{cmd}

## Step 4: Create Tissue/Cell-Type Specific Models

In [ ]:
# Load the output of step 1, which is a dictionary that specifies the merged list of active Gene IDs for each tissue
step1_results_file = os.path.join(configs.datadir, 'results', 'step1_results_files.json')
with open(step1_results_file) as json_file:
    context_gene_exp = json.load(json_file)

In [ ]:
# create tissue specific model, the names of output files are stored in dictionary tissue_spec_model

general_model_file = "GeneralModelUpdatedV2.mat"

recon_algorithm = 'IMAT' # troppo reconstruction algorithm to use
solver = "GUROBI"
output_filetypes = "xml mat json"

objective_dict = {
    "naiveB": 'biomass_maintenance',
    "smB": "biomass_reaction"
}

low_thresh = -5
high_thresh = -3

rev_dict = dict(reversed(context_gene_exp.items()))

for key,value in rev_dict.items():

    objective = objective_dict[key]
    active_genes_file = re.split('/|\\\\', value)[-1]
    genes_zscore_file = os.path.join(configs.datadir, "results", key, f"model_scores_{key}.csv")

    if recon_algorithm.upper() in ["IMAT", "TINIT"]:
        active_genes_filepath = os.path.join(configs.datadir, "results", key, genes_zscore_file)
    else:
        active_genes_filepath = os.path.join(configs.datadir, "results", key, active_genes_file)
        
    general_model_filepath = os.path.join(configs.datadir, general_model_file)
    boundary_rxns_filepath = os.path.join(configs.datadir, "boundary_rxns", "bcell_boundary_rxns.csv")
    force_rxns_filepath = os.path.join(configs.datadir, "force_rxns", "bcell_force_rxns.csv")

    cmd = ' '.join(
        [
            'python3', 'create_context_specific_model.py',
            '--context-name', f'"{key}"',
            '--reference-model-file', f'"{general_model_filepath}"',
            '--active-genes-filepath', f'"{active_genes_filepath}"',
            '--objective', f'"{objective}"',
            '--boundary-reactions-filepath', f'"{boundary_rxns_filepath}"',
            #'--exclude-reactions-filepath', f'"{exclude_rxns_file}"',
            '--force-reactions-filepath', f'"{force_rxns_filepath}"',
            '--algorithm', f'"{recon_algorithm}"',
            '--low-threshold', f'"{low_thresh}"',
            '--high-threshold', f'"{high_thresh}"',
            '--solver', f'"{solver}"',
            '--output-filetypes', f'"{output_filetypes}"'
        ]
    )

    !{cmd}


## Optional: Generate Memote Reports

In [ ]:
from escher import Builder
import cobra
import pandas as pd
import os
from project import configs

map_dir = os.path.join(
    configs.datadir,
    "local_files",
    "maps",
    "community-maps",
    "RECON1"
)

map_dict = {
    "trypto": os.path.join(map_dir, "RECON1.Tryptophan metabolism.json"),
    "lipid": os.path.join(map_dir, "RECON1.LIPID_METABOLISM.json"),
    "retinol": os.path.join(map_dir, "RECON1.Inositol retinol metabolism.json"),
    "glyco": os.path.join(map_dir, "RECON1.Glycolysis_TCA_PPP.json"),
    "combined": os.path.join(map_dir, "RECON1.COMBINED.json"),
    "carbo": os.path.join(map_dir, "RECON1.CARBOHYDRATE_METABOLISM.json"),
    "amino": os.path.join(map_dir, "RECON1.Amino acid metabolism (partial).json")
}


for context in ["naiveB", "smB"]:
    print(f"Starting {context}")
    for algorithm in ["IMAT"]:
        model_json=os.path.join(
            configs.datadir,
            "results",
            context,
            f"{context}_SpecificModel_{algorithm}.json"
        )

        model = cobra.io.load_json_model(model_json)
        for key in map_dict.keys():
            print(f"Running with: {key}")
            builder = Builder(map_json=map_dict[key])
            builder.model = model
            solution = cobra.flux_analysis.pfba(model)
            builder.reaction_data  = solution.fluxes
            #print(builder.fluxes.fluxes)
            #builder.fluxes = pd.read_csv(os.path.join(configs.datadir, "results", context, "iMAT_flux.csv"))["flux"].tolist()
            builder.reaction_scale = [
                { 'type': 'min', 'color': '#ff3300', 'size': 12 },
                { 'type': 'q1', 'color': '#ffc61a', 'size': 14 },
                { 'type': 'median', 'color': '#ffe700', 'size': 16 },
                { 'type': 'q3', 'color': '#4ffd3c', 'size': 18 },
                { 'type': 'max', 'color': '#3399ff', 'size': 20 }
            ]
            #builder.highlight_missing = TrueYeah I am still working on them. Cannot figure out a way to make them not contain a huge number of reactions though

            builder.reaction_no_data_color = "#8e8e8e"
            builder.save_html(os.path.join(
                configs.datadir,
                "results",
                context,
                "figures",
                f"{key}_map_{context}_{algorithm}.html"
            ))



In [ ]:
# generate memote reports for each  model
import os

for key,value in context_gene_exp.items():
    out_dir = os.path.join(
            configs.datadir, 
            "results",
            key
    )
    for algo in ["GIMME", "IMAT", "FASTCORE"]:
        report_file = os.path.join(out_dir, f"memote_report_{key}_{algo}.html")
        model_file = os.path.join(out_dir, f"{key}_SpecificModel_{algo}.xml")
        log_dir = os.path.join(out_dir, "memote")
        log_file = os.path.join(log_dir, f"{key}_{algo}_memote.log")

        if not os.path.exists(log_dir):
            os.mkdir(log_dir)

        cmd = ' '.join(
            [
                'memote',
                'report',
                'snapshot',
                '--filename',
                f'"{report_file}"', f'"{model_file}"', ">", f'"{log_file}"'
            ]
        )

        !{cmd}

## Step 5: Identifying disease related genes by analyzing transcriptomics data of patients
Differential Expression Analysis

In the config_sheets folder, there should be a folder called "disease". You can add a spreadsheet for each cell/tissue type called `disease_data_inputs_<tissue_name>`. Each sheet of this file should correspond to a seperate disease to analyze using DGE nfor that tissue. The source data can be either microarray or bulk RNA-seq and is formatted the same as if creating the base tissue model. The sheet names should contain the disease name, an underscore, and than either "microarray" or "bulk" depending on the source data. For example, if the disease is lupus, and the source data is bulk RNA-seq, the name of the sheet should be "lupus_bulk". This can be seen in the example sheet. If using bulk RNA-seq data, there should be a count matrix file in `/work/data/data_matrices/<tissue_name>/disease/` called `BulkRNAseqDataMatrix_<disease name>_<tissue name>`. 

*** Specify input files for step 3 here ***

In [ ]:
# specify tissue names to perform a disease analysis on. The diseases to analyze should be
# specified in `/work/data/config_sheets/disease/diease_data_inputs_<tissue name>`
context_names = ['naiveB', 'smB']
disease_names = ['arthritis', 'lupus_a', 'lupus_b']
data_source = 'rnaseq'

In [ ]:
# Differential gene expression analysis
taxon_id = "human"
for context_name in context_names:
    disease_config_file = f"disease_data_inputs_{context_name}.xlsx"
    cmd = ' '.join(
        [
            'python3', 'disease_analysis.py',
            '--context-name', f'"{context_name}"',
            '--config-file', f'"{disease_config_file}"',
            '--data-source', f'"{data_source}"',
            '--taxon-id', f'"{taxon_id}"'
        ]
    )

    !{cmd}

## Step 6: Identification of drug targets and repurposable drugs
This step maps drug targets in metabolic models,prforms knock out simulation, and compare simulation results with disease genes and identifies drug targets and repurposable drugs

*** Specify input files for step 4 here ***

1. Instruction: A processed Drug-Target file is included in the `/root/pipelines/data/`. (Optional step) For the updated versions the users can download `Repurposing_Hub_export.txt` from [Drug Repurposing Hub](https://clue.io/repurposing-app). From the downloaded file first remove all the activators, agonists, and withdrawn drugs and then upload to to `/root/pipelines/data/`.

2. To use automatically created tissue specific models. Note: It is recommended to use refined and validated models for further analysis. User can define cutomized models in next sub-step.

3. To use customized model, please specify `tissue_spec_model` manually, e.g. uncomment tissue_spec_model in the following cell.

In [ ]:
# Knock out simulation for the analyzed tissues and diseases

# Define custom models to use in drug repurposing
# "context_name" should match one of context_names from Step 5
# Right-click the model name in the file viewer (on the left), and select "copy path"
# Make sure a leading "/" is present at the beginning of the file path
model_files = {
    # "context_name": "/path/to/model.mat"
    # EXAMPLE -> "Treg": "/home/jovyan/work/data/results/naiveB/naiveB_SpecificModel_IMAT.mat"
}

diseases = ["arthritis", "lupus_a", "lupus_b"]
methods = ["IMAT"]

for context in context_names:
    for method in methods:
        for disease in diseases:

            disease_path = os.path.join(configs.datadir, "results", context, disease)
            out_dir = os.path.join(configs.datadir, "results", context, disease)
            tissue_gene_folder = os.path.join(configs.datadir, context)
            os.makedirs(tissue_gene_folder, exist_ok=True)
            inhibitors_file = f'{context}_inhibitors_Entrez.txt'

            if not os.path.exists(disease_path):
                continue

            # load the results of step 3 to dictionary 'disease_files'
            step3_results_file = os.path.join(
                configs.datadir,
                'results',
                context,
                disease,
                'step2_results_files.json'
            )

            with open(step3_results_file) as json_file:
                disease_files = json.load(json_file)

            Disease_Down = disease_files['DN_Reg']
            Disease_Up = disease_files['UP_Reg']
            drug_raw_file = 'Repurposing_Hub_export.txt'

            # Test if the user has specified a model for this cell type
            # If they have not specified a model, use the automatically generated one
            if context in model_files.keys():
                tissueSpecificModelfile = model_files[context]
            else:
                tissueSpecificModelfile  = os.path.join(
                    configs.datadir,
                    "results",
                    context,
                    f"{context}_SpecificModel_{method}.mat"
                )

            if method == "IMAT":
                ref_flux_file = os.path.join(
                    configs.datadir,
                    "results",
                    context,
                    "IMAT_flux.csv"
                )
                cmd = ' '.join(
                    [
                        'python3' , 'knock_out_simulation.py',
                        '--context-model', f'"{tissueSpecificModelfile}"',
                        '--context-name', f'"{context}"',
                        '--disease-name', f'"{disease}"',
                        '--disease-up', f'"{Disease_Up}"',
                        '--disease-down', f'"{Disease_Down}"',
                        '--raw-drug-file', f'"{drug_raw_file}"',
                        '--reference-flux-file', f'"{ref_flux_file}"',
                        #'--test-all'
                    ]
                )

                !{cmd}


            else:
                cmd = ' '.join(
                    [
                        'python3' , 'knock_out_simulation.py',
                        '--context-model', f'"{tissueSpecificModelfile}"',
                        '--context-name', f'"{context}"',
                        '--disease-name', f'"{disease}"',
                        '--disease-up', f'"{Disease_Up}"',
                        '--disease-down', f'"{Disease_Down}"',
                        '--raw-drug-file', f'"{drug_raw_file}"',
                        #'--test-all'
                    ]
                )

                !{cmd}


In [ ]:
map_dir = os.path.join(
    configs.datadir,
    "local_files",
    "maps",
    "community-maps",
    "RECON1"
)

map_dict = {
    "trypto": os.path.join(map_dir, "RECON1.Tryptophan metabolism.json"),
    "lipid": os.path.join(map_dir, "RECON1.LIPID_METABOLISM.json"),
    "retinol": os.path.join(map_dir, "RECON1.Inositol retinol metabolism.json"),
    "glyco": os.path.join(map_dir, "RECON1.Glycolysis_TCA_PPP.json"),
    "combined": os.path.join(map_dir, "RECON1.COMBINED.json"),
    "carbo": os.path.join(map_dir, "RECON1.CARBOHYDRATE_METABOLISM.json"),
    "amino": os.path.join(map_dir, "RECON1.Amino acid metabolism (partial).json")
}

targets_dict = {
    "SLC2A1": "6513",
    "SLC2A3": "6515",
    "HPRT1": "3251",
    "SLC10A1": "6554",
    "DHODH": "1723"
}

tests_arthritis = [
    "arthritis_imat_neg5_neg3",
    "arthritis_imat_neg5_2",
]

tests_lupus = [
    "lupus_a_imat_neg5_neg3",
    "lupus_a_imat_neg5_2",
    "lupus_b_imat_neg5_neg3",
    "lupus_b_imat_neg5_2",
]

In [ ]:
def get_test_results(contexts, tests, targets_dict):
    for context in contexts:
        for test in tests:
            if "imat" in test.split("_"):
                params = "_".join(test.split("_")[-2:])
                algo = "IMAT"
            else:
                params = "_".join(test.split("_")[-4])
                algo = "GIMME"

            test_dir = os.path.join(configs.datadir, "results", context, test)
            if os.path.exists(test_dir):
                ratio_file = os.path.join(test_dir, "flux_ratios_KO.csv")
                ratio_df = pd.read_csv(ratio_file)
                for targ, entrez in targets_dict.items():
                    targ_flux = ratio_df[entrez]
                    model_json=os.path.join(
                        configs.datadir, 
                        "results", 
                        context,
                        f"{context}_SpecificModel_{algo}_{params}.json"
                    )

                    model = cobra.io.load_json_model(model_json)
                    for m in map_dict.keys():
                        map_test_dir = os.path.join(
                            configs.datadir,
                            "results",
                            context,
                            "figures",
                            "maps",
                            test
                        )
                        if not os.path.exists(map_test_dir):
                            os.makedirs(map_test_dir)
                        map_targ_dir = os.path.join(map_test_dir, targ)
                        if not os.path.exists(map_targ_dir):
                            os.makedirs(map_targ_dir)   
                        builder = Builder(map_json=map_dict[m])
                        builder.model = model
                        #solution = cobra.flux_analysis.pfba(model)
                        #builder.reaction_data  = solution.fluxes
                        builder.reaction_data = targ_flux
                        #print(builder.fluxes.fluxes)
                        #builder.fluxes = pd.read_csv(os.path.join(configs.datadir, "results", context, "iMAT_flux.csv"))["flux"].tolist()
                        builder.reaction_scale = [
                            { 'type': 'value', value: 0.5, 'color': '#ffffbf', 'size': 10 },
                            { 'type': 'value', value: 0.9, 'color': '#ffffbf', 'size': 10 },
                            { 'type': 'value', value: 1.0, 'color': '#ffffbf', 'size': 10 },
                            { 'type': 'value', value: 1.1, 'color': '#ffffbf', 'size': 10 },
                            { 'type': 'value', value: 2.0, 'color': '#ffffbf', 'size': 10 }
                        ]
                        #builder.highlight_missing = TrueYeah I am still working on them. Cannot figure out a way to make them not contain a huge number of reactions though

                        builder.reaction_no_data_color = "#8e8e8e"
                        builder.save_html(
                            os.path.join(
                                map_targ_dir,
                                f"{m}_map_{context}_{test}_{targ}.html"
                            )
                        )                  

In [ ]:
import escher
from escher import Builder
import cobra
import pandas as pd
import os
from project import configs

In [ ]:
test = "arthritis_imat_neg5_neg3"
context = "naiveB"
if "imat" in test.split("_"):
    params = "_".join(test.split("_")[-2:])
    algo = "IMAT"
else:
    params = "_".join(test.split("_")[-4])
    algo = "GIMME"
print("param: ", params)
test_dir = os.path.join(configs.datadir, "results", context, test)
if os.path.exists(test_dir):
    ratio_file = os.path.join(test_dir, "flux_ratios_KO.csv")
    ratio_df = pd.read_csv(ratio_file)
    targ = "SLC2A1"
    entrez = "6513"
    targ_flux = ratio_df[entrez]
    model_json=os.path.join(
        configs.datadir, 
        "results", 
        context,
        f"{context}_SpecificModel_{algo}_{params}.json"
    )
    print(model_json)
    #model = cobra.io.load_json_model(model_json)
    m = 'glyco'
    map_test_dir = os.path.join(
        configs.datadir,
        "results",
        context,
        "figures",
        "maps",
        test
    )
    if not os.path.exists(map_test_dir):
        os.makedirs(map_test_dir)
    map_targ_dir = os.path.join(map_test_dir, targ)
    if not os.path.exists(map_targ_dir):
        os.makedirs(map_targ_dir)   

In [ ]:
#model = cobra.io.load_json_model(model_json)
builder = Builder(map_json=map_dict[m])
#builder.model = model
builder.model_json = model_json

In [ ]:
if 'builder' in locals(): del builder
#solution = cobra.flux_analysis.pfba(model)
#builder.reaction_data  = solution.fluxes
rxns = [rxn.id for rxn in model.reactions]
targ_flux.index = rxns
#targ_flux.fillna(-9999,inplace=True)
print(list(np.where(np.isnan(targ_flux.tolist()))))
targ_flux.replace(np.inf, 9999, inplace=True)
targ_flux.replace(-np.inf, -9999, inplace=True)
targ_flux.replace(np.nan, 1.0 , inplace=True) # nans should be 1.0
print(targ_flux)
bad_f = []
print(targ_flux.tolist()[117])
print(type(targ_flux.tolist()[117]))
for f in targ_flux.tolist():
    bad_f.append(type(f))
print(set(bad_f))
    
#builder.reaction_data = targ_flux
#builder.reactions_styles=['color', 'style', 'text']
#print(builder.fluxes.fluxes)
#builder.fluxes = pd.read_csv(os.path.join(configs.datadir, "results", context, "iMAT_flux.csv"))["flux"].tolist()
rxn_scale = [
    { 'type': 'value', 'value': -9999, 'color': '#000000', 'size': 10 },
    { 'type': 'value', 'value': 0.5, 'color': '#2c7bb6', 'size': 10 },
    { 'type': 'value', 'value': 0.9, 'color': '#abd9e9', 'size': 10 },
    { 'type': 'value', 'value': 1.0, 'color': '#ffffbf', 'size': 10 },
    { 'type': 'value', 'value': 1.1, 'color': '#fdae61', 'size': 10 },
    { 'type': 'value', 'value': 2.0, 'color': '#d7191c', 'size': 10 },
    { 'type': 'value', 'value': 9999, 'color': '#c90076', 'size': 10 },
    { 'type': 'value', 'value': -500, 'color': '#8e8e8e', 'size': 10 }
    
]
builder = Builder(
    model = model,
    map_json=map_dict[m],

    reaction_data = targ_flux,
    reaction_scale = rxn_scale,
    highlight_missing = True,
    reaction_no_data_color = '#8e8e8e',
    reaction_styles = ['color', 'style', 'text']
)
#builder.highlight_missing = True
builder.reaction_no_data_color = "#8e8e8e"
builder

In [ ]:
map_file = os.path.join(
     map_targ_dir,
    f"{m}_map_{context}_{test}_{targ}.html"
)
print(map_file)
builder.save_html(os.path.join(map_targ_dir, f"{m}_map_{context}_{test}_{targ}.html"))

In [ ]:
get_test_results(context_names, tests_arthritis, targets_dict)
